Mini-projet XAI — **ProtoPNet (Prototypical Part Network)**
"This Looks Like That: Deep Learning for Interpretable Image Recognition"

**Étudiant**: CHAIBOU SAIDOU Abdoulaye<br>
**Professeur**: Tawfik Masrour<br>
**Date**: 16 Décembre 2025<br>
**Filière**: IATD-SI<br>
**Etablissement**: [ENSAM-Meknès](https://ensam.umi.ac.ma)

**Remerciements**:
Ce projet est basé sur l'article "This Looks Like That: Deep Learning for 
Interpretable Image Recognition" (NeurIPS 2019), par Chaofan Chen*, Oscar Li*, 
Chaofan Tao, Alina Jade Barnett, Jonathan Su, et Cynthia Rudin (Duke University).
Code original: [PPNet repo](https://github.com/cfchen-duke/ProtoPNet).

**Infos**: Ce notebook présente brièvement la technique interprétable d'explication des réseaux de convolutifs appelée Prototypical Part Network par ses auteurs officiels, sous forme de mini-projet (IA Explicable) de classe. Nous utilisons egalement un modèle pré-entraîné par *Alina Jade Barnett* pour l'analyse de la methode, par soucis de simplicité et de gain de temps.

- [Introduction](#introduction)
- [Explication globale de la methode](#explication)
- [Exemple](#exemple)
- [Références](#références)

## Introduction

**LE PROBLÈME DES BOÎTES NOIRES** :

Les réseaux de neurones profonds (CNN) sont extrêmement performants pour la
classification d'images, MAIS ils sont des "boîtes noires" inexplicables :

  Médecin : "Pourquoi le modèle dit que c'est un cancer ?"
  CNN Classique : "🤷 Résultat = 95% cancer"
  
  Juge : "Sur quelle base le modèle a identifié cette personne ?"
  CNN Classique : "🤷 Confiance = 98%"

❌ Conséquences :
  - Impossible de vérifier le raisonnement
  - Risque de biais cachés
  - Manque de confiance des experts
  - Non-conforme aux régulations (RGPD, FDA)

💡 LA SOLUTION PROTOPNET :

Au lieu d'une boîte noire, ProtoPNet fonctionne comme l'esprit humain :
raisonnement par SIMILITUDE avec des CAS MÉMORISÉS (prototypes).

  Médecin : "Pourquoi cancer ?"
  ProtoPNet : "Cette région ressemble à 85% à ce prototype de mélanome,
               cette autre région ressemble à 78% à cet autre prototype..."
  Médecin : ✅ "Je peux vérifier ! Le raisonnement est correct."


## Explication globale de la methode
### Type d'explication

📊 GLOBALE + LOCALE :
  - GLOBALE : Les prototypes sont appris une fois pour toutes les classes
  - LOCALE : Chaque prédiction s'explique par similarité aux prototypes

🖼️ DOMAINE D'APPLICATION : IMAGES / CNN
  - Classification d'oiseaux (dataset CUB-200)
  - Diagnostic médical (dermatologie, radiologie)
  - Contrôle qualité industriel
  - Toute tâche de classification fine-grained

🎯 AVANTAGE CLÉ : INTERPRÉTABILITÉ INTRINSÈQUE

Contrairement à LIME, SHAP, Grad-CAM (méthodes POST-HOC qui approximent),
ProtoPNet est INTRINSÈQUEMENT INTERPRÉTABLE :
  - L'explication fait partie de l'architecture
  - Pas d'approximation
  - Fidélité garantie à 100%

### ANALOGIE HUMAINE

Imaginez que vous apprenez à reconnaître des oiseaux :

1️⃣ PHASE D'APPRENTISSAGE (mémorisation de prototypes) :
   
   Vous voyez beaucoup d'oiseaux et mémorisez des CARACTÉRISTIQUES TYPIQUES :
   ```
   🐦 Cardinals :
      → Prototype A : "crête rouge vif caractéristique"
      → Prototype B : "bec orange pointu"
      → Prototype C : "masque noir autour des yeux"
   
   🐦 Geais bleus :
      → Prototype D : "plumes bleues brillantes"
      → Prototype E : "bande noire sur la tête"
      → Prototype F : "bout des ailes blanc"
```
2️⃣ PHASE DE RECONNAISSANCE (nouvelle image) :
   
   Vous voyez un nouvel oiseau inconnu :
   
   
   | 🐦 Oiseau mystère |
|------------------|
| ✓ Cette partie ressemble beaucoup au **Prototype A** |
| ✓ Cette autre partie ressemble au **Prototype B** |
| ✗ Pas de similarité avec les prototypes de **geai bleu** |

   
   CONCLUSION : "C'est probablement un cardinal !"

3️⃣ EXPLICATION :
   
   Si on vous demande "Pourquoi cardinal ?" :
   ```
   → "Parce que sa crête ressemble au Prototype A (85% similaire)"
   → "Et son bec ressemble au Prototype B (78% similaire)"
   ```

### C'EST EXACTEMENT CE QUE FAIT PROTOPNET !

🏗️ ARCHITECTURE EN 3 ÉTAPES :
```
   ┌─────────────────┐
   │  Image Input    │  📸 Exemple : photo d'oiseau 224×224×3
   │  (224×224×3)    │
   └────────┬────────┘
            │
            ▼
   ┌─────────────────┐
   │  CNN Backbone   │  🔧 VGG16 / ResNet : extraction de features
   │  (ex: VGG16)    │     Sortie : feature map H×W×D
   └────────┬────────┘     (ex: 14×14×128)
            │
            ▼
   ┌─────────────────┐
   │ Prototype Layer │  🎯 Compare chaque région aux m prototypes
   │  (m prototypes) │     Calcule similarités spatiales
   └────────┬────────┘     Sortie : m scores de similarité
            │
            ▼
   ┌─────────────────┐
   │Classification   │  📊 Combine les similarités avec poids appris
   │     Layer       │     Sortie : scores par classe
   └─────────────────┘
```
![diagram of the architecture](images/architectureppnet.png)
📤 QUE PRODUIT LA MÉTHODE ?

Pour chaque image à classifier :

1. HEATMAP DE SIMILARITÉ:
```
 pour chaque prototype activé → Montre QUELLE RÉGION de l'image ressemble au prototype
 ```

2. TOP-K PROTOTYPES:
```
les plus similaires → "Les 5 prototypes qui matchent le mieux avec cette image"
```
3. EXPLICATION VISUELLE :
   → "Cette partie [bbox rouge] ressemble à 85% au Prototype #42"
   → "Prototype #42 = crête rouge d'un cardinal"

4. RAISONNEMENT COMPLET :
```
   → Similarité prototype 1 : 0.85  ×  Poids 1.2  =  +1.02
   → Similarité prototype 2 : 0.78  ×  Poids 0.9  =  +0.70
   → Similarité prototype 3 : 0.12  ×  Poids -0.3 =  -0.04
   → ──────────────────────────────────────────────────
   → TOTAL score pour "Cardinal" : 1.68 → Softmax → 92%
```
🎯 FAMILLE XAI :
Comparaison entre ProtoPNet (interprétabilité intrinsèque) et les méthodes post-hoc courantes (LIME, SHAP, Grad-CAM) :

| Aspect | ProtoPNet (interprétabilité intrinsèque) | Méthodes post-hoc (LIME, SHAP, Grad-CAM) |
|------|------------------------------------------|------------------------------------------|
| Moment de l’explication | **Pendant la prédiction** | Après la prédiction |
| Principe | Raisonnement par **similarité avec des prototypes** | Approximation ou analyse a posteriori |
| Lien avec le modèle | L’explication **fait partie du modèle** | L’explication est **ajoutée par-dessus** |
| Fidélité à la décision | **100 % fidèle** (pas d’approximation) | Approximative |
| Type d’explication | « *Cette partie ressemble à ce prototype* » | Importance de pixels / features |
| Raisonnement | **Case-Based Reasoning** (par cas) | Attribution de contribution |
| Stabilité | Stable (mêmes prototypes) | Peut varier selon les paramètres |
| Coût de calcul | Modéré | SHAP souvent **très coûteux** |
| Compréhensible par humain | **Oui (visuel + sémantique)** | Parfois difficile à interpréter |
| Confiance utilisateur | Élevée | Variable |


### NOTATIONS

- $x ∈ ℝ^{224\times224\times3}$     : image d'entrée RGB
- $f(x) ∈ ℝ^{H\times\text{W}\times\text{D}}$       : carte de features extraite par le CNN
                          (ex: H=14, W=14, D=128 pour input $224\times224$)
- $P = {p₁, ..., pₘ}$     : ensemble des m prototypes
                          (chaque $p_j ∈ ℝ^D$, même dimension que features)
- $d(·,·)$                : distance L2 (euclidienne)
- $K$                     : nombre de classes

### ÉQUATION — SIMILARITÉ ENTRE UNE IMAGE ET UN PROTOTYPE (ProtoPNet)

Soit une image d’entrée $x \in \mathbb{R}^{224 \times 224 \times 3}$.
Les couches convolutionnelles du réseau extraient une représentation latente :

$$
f(x) \in \mathbb{R}^{H \times W \times D}
$$

Dans le cas du dataset CUB-200-2011 :
$$
H = W = 7, \quad D \in \{128, 256, 512\}
$$

Le réseau apprend un ensemble de $m$ prototypes :
$$
P = \{p_j\}_{j=1}^{m}, \quad
p_j \in \mathbb{R}^{H_1 \times W_1 \times D}
$$

Dans les expériences ProtoPNet :
$$
H_1 = W_1 = 1
$$

Chaque prototype $p_j$ représente donc un **motif latent local**, correspondant à une **partie prototypique de l’objet** dans l’image (ex. : tête d’oiseau, aile, plumage).

---

#### Calcul de la similarité à un prototype

Pour un prototype donné $p_j$, on calcule sa similarité avec toutes les régions spatiales de la carte de caractéristiques $f(x)$.

#### 1. Distance L2 au carré

Pour chaque position spatiale $(h,w)$ :

$$
d^2\!\left(f(x)_{h,w},\, p_j\right)
=
\left\| f(x)_{h,w} - p_j \right\|_2^2
=
\sum_{k=1}^{D}
\left( f(x)_{h,w,k} - p_j[k] \right)^2
$$

où :
- $f(x)_{h,w} \in \mathbb{R}^D$ est le vecteur de caractéristiques local,
- $p_j \in \mathbb{R}^D$ est le prototype.

---

#### 2. Conversion distance → similarité

La distance est transformée en score de similarité par une fonction exponentielle décroissante :

$$
g_{p_j}(z)
=
\max_{\tilde{z} \in \mathrm{patches}(z)}
\;
\log\!\left(
\frac{
\|\tilde{z} - p_j\|_2^2 + 1
}{
\|\tilde{z} - p_j\|_2^2 + \varepsilon
}
\right)
$$

$$
\textbf{avec :}
$$
$$
z = f(x)
\quad \text{: sortie des couches convolutionnelles}
$$

$$
\tilde{z}
\quad \text{: patch latent de } z \text{ de même dimension que le prototype } p_j
$$

$$
d^2 = \|\tilde{z} - p_j\|_2^2
\quad \text{: distance euclidienne au carré (L2)}
$$

Cette fonction garantit que :
- Quand $d^2 \to 0$ :
$$
g_{p_j}(z) \;\longrightarrow\; \log\left(\frac{1}{\varepsilon}\right)
\quad \text{(grande similarité)}
$$
- Quand $d^2 \to \infty$ :
$$
g_{p_j}(z) \;\longrightarrow\; \log(1) = 0
\quad \text{(aucune similarité)}
$$

---

#### 3. Agrégation spatiale (Global Max Pooling)

Le score de similarité final entre l’image $x$ et le prototype $p_j$ est obtenu par :

$$
\boxed{
\mathrm{sim}(x, p_j)
=
\max_{h,w}
\;
\exp\!\left(
- d^2\!\left(f(x)_{h,w}, p_j\right)
\right)
}
$$

Cette opération sélectionne la **région de l’image la plus proche du prototype** dans l’espace latent.

---

#### Intuition

$$
\text{``Quelle est la région de l’image qui ressemble le plus au prototype } p_j \text{ ?''}
$$

- Si $\mathrm{sim}(x, p_j)$ est élevé :
  - il existe une région de l’image très proche du prototype,
  - le concept représenté par $p_j$ est fortement présent.
- Sinon :
  - le prototype n’est pas pertinent pour cette image.

---

#### Rôle dans la prédiction finale

Chaque classe $k \in \{1,\dots,K\}$ est associée à un sous-ensemble de prototypes $P_k \subseteq P$.
Les scores de similarité produits par tous les prototypes sont ensuite combinés par une couche linéaire :

$$
\text{logits}(x) = W_h \cdot
\begin{bmatrix}
\mathrm{sim}(x,p_1) \\
\vdots \\
\mathrm{sim}(x,p_m)
\end{bmatrix}
$$

puis normalisés par une fonction softmax pour produire les probabilités de classification.

---

**ProtoPNet peut ainsi être interprété comme un modèle prenant ses décisions en comparant explicitement l’image à des prototypes visuels interprétables.**


##### Exemple concret

###### Classe **"Cardinal"** ($ k = 5 $)

- **Prototype 25** (crête rouge)  
  $$ \mathrm{sim} = 0.85, \quad w_{5,25} = 1.2 \quad \Rightarrow \quad +1.02 $$

- **Prototype 26** (bec orange)  
  $$ \mathrm{sim} = 0.78, \quad w_{5,26} = 0.9 \quad \Rightarrow \quad +0.70 $$

- **Prototype 27** (masque noir)  
  $$ \mathrm{sim} = 0.65, \quad w_{5,27} = 0.7 \quad \Rightarrow \quad +0.46 $$

- **Autres 97 prototypes**  
  contribution faible  
  $$ \Rightarrow +0.32 $$

---

###### Score total

$$
g_5(x) = 2.50
\quad \Rightarrow \quad
P(\text{Cardinal} \mid x) = 0.92
$$


### Algorithme d'entraînement de ProtoPNet

L'entraînement de **ProtoPNet** est structuré en **trois étapes principales**, pouvant être répétées plusieurs fois :

1. **Optimisation par descente de gradient stochastique (SGD)** des couches convolutionnelles et des prototypes  
2. **Projection des prototypes** sur des patches latents réels  
3. **Optimisation convexe de la dernière couche fully-connected**

L'objectif global est d'apprendre un **espace latent interprétable**, où chaque prototype représente un concept visuel discriminant pour une classe donnée.


#### Étape 1 — Apprentissage de l’espace latent (SGD)

Dans cette première phase, on apprend :
- les paramètres des couches convolutionnelles $w_conv$
- les prototypes $P = {p_j}$

La dernière couche de classification $h$ est **figée** afin d'imposer une structure sémantique dans l’espace latent.

$$
\min_{P, w_{\text{conv}}}
\frac{1}{n} \sum_{i=1}^{n}
\text{CrsEnt}(h \circ g_p \circ f(x_i), y_i)
+ \lambda_1 \, \text{Clst}
+ \lambda_2 \, \text{Sep}
$$


La fonction de perte est composée de trois termes :

- **Cross-Entropy** : pénalise les erreurs de classification
- **Coût de clustering (Clst)** : rapproche les patches latents d’une image vers les prototypes de sa classe
- **Coût de séparation (Sep)** : éloigne les patches latents des prototypes des autres classes

$$
\text{Clst} =
\frac{1}{n} \sum_{i=1}^{n}
\min_{j : p_j \in P_{y_i}}
\min_{z \in \text{patches}(f(x_i))}
\| z - p_j \|_2^2
$$

$$
\text{Sep} =
-\frac{1}{n} \sum_{i=1}^{n}
\min_{j : p_j \notin P_{y_i}}
\min_{z \in \text{patches}(f(x_i))}
\| z - p_j \|_2^2
$$

#### Rôle du coût de séparation

Le terme **Sep**, introduit dans ProtoPNet, empêche un prototype d'une classe donnée
de représenter un concept visuel présent dans d'autres classes.

Cela améliore la **discrimination inter-classes** et renforce l’interprétabilité.

#### Fixation de la dernière couche pendant le SGD

La matrice de poids $w_h$ de la couche finale est fixée comme suit :

- Connexion **positive** entre un prototype et sa classe
- Connexion **négative** avec les autres classes

Cela force le réseau à apprendre des prototypes réellement spécifiques à leur classe.

$$
w_h^{(k,j)} =
\begin{cases}
1 & \text{si } p_j \in P_k \\
-0.5 & \text{sinon}
\end{cases}
$$

#### Étape 2 — Projection des prototypes

Après l’apprentissage, chaque prototype est projeté sur le **patch latent réel**
le plus proche appartenant à sa classe.

Cela permet d’associer chaque prototype à une **région visuelle concrète** d’une image d’entraînement.

$$
p_j \leftarrow
\arg\min_{z \in Z_j}
\| z - p_j \|_2^2
\quad
\text{avec }
Z_j = \{ z \in \text{patches}(f(x_i)) \mid y_i = k \}
$$


#### Interprétation

Après projection :
- chaque prototype correspond à **un patch d’image réel**
- la visualisation devient directe et humaine
- l’interprétabilité du modèle est maximale

$$
\Delta_{\max} = m \log\big((1+\delta)(2-\delta)\big)
$$
$$
\text{Si } 
\text{logit}_{\text{top1}} - \text{logit}_{\text{top2}}
\ge 2 \Delta_{\max}
\Rightarrow \text{classe inchangée}
$$

#### Étape 3 — Optimisation convexe de la dernière couche

Dans cette dernière phase :
- les couches convolutionnelles et prototypes sont figées
- seuls les poids $w_h$ sont optimisés

L’objectif est de **réduire les connexions négatives inutiles**.

$$
\min_{w_h}
\frac{1}{n} \sum_{i=1}^{n}
\text{CrsEnt}(h \circ g_p \circ f(x_i), y_i)
+ \lambda \sum_{k=1}^{K}
\sum_{j : p_j \notin P_k}
| w_h^{(k,j)} |
$$

#### Intuition finale

Cette étape :
- améliore la précision
- favorise un raisonnement **positif** ("cette image ressemble à…")
- évite le raisonnement négatif ("ce n’est pas cette classe")

L’espace latent et les prototypes restent inchangés.

### Projection des prototypes

Afin de rendre les prototypes **visuellement interprétables**, chaque prototype est projeté
(« push ») vers un **patch latent réel** provenant des images d’entraînement de **la même classe**.

Après projection, chaque prototype peut être assimilé à un **patch d’image concret**, ce qui
permet une visualisation directe et une interprétation humaine du raisonnement du réseau.

#### Principe

- Les prototypes sont appris dans l’espace latent
- Ils ne correspondent pas nécessairement à des patches réels
- La projection remplace chaque prototype par le patch latent réel **le plus proche**
- La classe du patch projeté est **identique** à celle du prototype
$$
p_j \leftarrow
\arg\min_{z \in Z_j}
\| z - p_j \|_2^2
$$
$$
Z_j =
\left\{
\tilde{z} \;\middle|\;
\tilde{z} \in \text{patches}(f(x_i)),
\; \forall i \text{ tel que } y_i = k
\right\}
$$
où :
- $p_j$ est un prototype appartenant à la classe $k$ (i.e. $p_j ∈ P_k$)
- $f(x_i)$ désigne la représentation latente de l’image $x_i$
- $\text{patches}(f(x_i))$ est l’ensemble des patches latents extraits
- $Z_j$ contient tous les patches latents des images d’entraînement de la classe $k$


#### Intuition géométrique

La projection correspond à un **déplacement minimal** du prototype dans l’espace latent,
vers un point réel du jeu d’entraînement.

Ainsi :
- la structure de l’espace latent est conservée
- les prototypes deviennent interprétables
- la classification reste stable si le déplacement est faible

#### Impact sur la classification

La projection pourrait, en théorie, modifier les distances entre prototypes et patches latents,
et donc affecter les logits de classification.

Le théorème suivant formalise les conditions sous lesquelles la projection **ne modifie pas**
la prédiction du modèle.

#### Notation pour l’analyse théorique

Pour l’analyse, on adopte une notation alternative :

- $p_l^k$ : le $l-ième$ prototype de la classe $k$
- $b_l^k$ : valeur du prototype **avant projection**
- $a_l^k$ : valeur du prototype **après projection**

On considère une image d’entrée $x$, correctement classifiée avant projection,
et on note :

- $z_l^k$ : le patch latent de $f(x)$ le plus proche du prototype $p_l^k$
- $c$ : la classe correcte de l’image $x$

Le théorème établit que, sous certaines hypothèses de **déplacement limité**
des prototypes et de **marge suffisante entre les logits**, la projection
des prototypes n’altère pas la prédiction finale du réseau.

### Visualisation des prototypes

Après la projection, chaque prototype $p_j$ correspond à un patch latent réel
provenant d’une image d’entraînement $x$.

L’objectif de cette étape est d’identifier **quelle région de l’image originale**
(en espace pixel) correspond à ce prototype afin de rendre le raisonnement du
modèle visuellement interprétable.

#### Principe général

Un prototype doit être visualisé à partir de la région de l’image
qui **l’active le plus fortement**.

L’hypothèse est la suivante :
> le patch de l’image correspondant à un prototype est celui
> pour lequel l’activation du prototype est maximale.

#### Calcul de la carte d’activation du prototype

Pour une image d’entraînement $x$ associée au prototype $p_j$ :

1. L’image $x$ est propagée dans le réseau ProtoPNet entraîné
2. Le prototype $p_j$ produit une **carte d’activation spatiale**
   via l’unité de prototype $g_{p_j}$
3. Cette carte est extraite **avant l’opération de max-pooling**

#### Remise à l’échelle spatiale (Upsampling)

La carte d’activation issue de $g_{p_j}$ est ensuite **redimensionnée**
à la taille de l’image originale $x$.

Cela permet d’associer chaque activation latente
à une région précise de l’image en espace pixel.

$$
A_j(x) = \text{Upsample}\left( g_{p_j}(f(x)) \right)
$$
où :
- $g_{p_j}$ désigne l’unité associée au prototype $p_j$
- $f(x)$ est la représentation latente de l’image
- $A_j(x)$ est la carte d’activation du prototype $p_j$
  à la résolution de l’image originale.


#### Sélection de la région la plus activée

La région visuelle associée au prototype est définie
comme la **plus petite boîte rectangulaire**
contenant les pixels fortement activés.

$$
\mathcal{R}_j =
\left\{
u \in x \;\middle|\;
A_j(x)[u] \ge \text{Percentile}_{95}(A_j(x))
\right\}
$$
Le prototype $p_j$ est alors visualisé à l’aide du **plus petit rectangle**
englobant tous les pixels de $𝓡_j$.

Ce choix du **95ᵉ percentile** permet :
- d’éliminer le bruit
- de se concentrer sur la région la plus discriminante
- d’obtenir une visualisation compacte et interprétable

#### Résumé de la visualisation des prototypes

Pour chaque prototype :

1. Propagation de l’image associée dans le réseau
2. Extraction de la carte d’activation du prototype
3. Upsampling à la taille de l’image
4. Seuil au 95ᵉ percentile
5. Extraction du patch visuel correspondant

Cette procédure permet de relier explicitement :
- un **prototype latent**
- un **concept visuel localisé**
- une **région interprétable de l’image**

Elle constitue un élément central de l’explicabilité
de ProtoPNet.

## Exemple

J'ai utilisé un modèle entrainé par une co-autrice de la méthode PPNet.
Vous pouvez trouver le modèle [ici](https://drive.google.com/drive/folders/1C6gpcmIBu_djO5cUXPgjXi1FqIWa3eI6?usp=sharing) il est car absent sur ce dépot github.

Pour plus d'infos comme comment entrainer votre propre modèle ou comment modifier l'architecture etc..., réferez-vous [ici](./ProtoPNet_README.txt).

Pour faire de la prediction, nous renseignons quelques informations dans le fichier analyse.ps1, à savoir:
- `-modeldir` : **dossier dans lequel le modèle d'annalyse est placé**
- `-model` : **nom du modele**
- `-imgdir` **dossier dans lequel l'image à annalyser est placé**
- `-img` : **nom de l'image**
- `-imgclass` : **L'indice de la classe de l'image, sachant les indices commencent par 0**.

voilà un appercu:
```ps1
python local_analysis.py `
-modeldir ./saved_models/1215_vgg16_retrain `
-model 20_9push0.7337.pth `
-imgdir ./images_test `
-img European_Goldfinch_0006_794661.jpg `
-imgclass 47
```

Voici l'image à tester<br>
![European_Goldfinch_0006_794661.jpg](images_test\European_Goldfinch_0006_794661.jpg)